In [2]:
#Convert pdf to txt
import PyPDF2
def ConvertPdf(i):
    pdfFileObj= open(i[1],'rb')
    pdfReader= PyPDF2.PdfFileReader(pdfFileObj)
    num_pages=pdfReader.numPages
    count=0
    text=""
    while count<num_pages:
        pageObj=pdfReader.getPage(count)
        count+=1
        text+=pageObj.extractText()
        text=text.replace('\n',' ')
        text=text.lower()
    return text

In [3]:
#Convert word to txt
import textract
def ConvertWord(i):
    wordtext=textract.process(i[1])
    wordtext=wordtext.replace(b'\n',b'')
    wordtext=wordtext.replace(b'\r',b'')
    wordtext=wordtext.replace(b'\t',b'')
    text1=str(wordtext)
    text1=text1.lower()
    return text1

In [4]:
## judge pdf/word+ connect
import os
import glob

    
list_of_files=[]
list_of_files_pdf=[]
list_of_files_doc=[]
list_of_files_docx=[]
resumes=[]
os.chdir('C:/Users/DELL-LH2018/Desktop/intern/All original resume')
for file in glob.glob('**/*.pdf',recursive=True):
    list_of_files_pdf.append(file)
for file in glob.glob('**/*.doc',recursive=True):
    list_of_files_doc.append(file)
for file in glob.glob('**/*.docx',recursive=True):
    list_of_files_docx.append(file)
list_of_files = list_of_files_pdf+list_of_files_doc+list_of_files_docx
print("This is LIST OF FILES")
print(list_of_files)
    
#convert
print("######## PARSING #########")
for i in enumerate(list_of_files):
    Temp= i[1].split(".")
    if Temp[1]=="pdf" or Temp[1]=="Pdf" or Temp[1]=="PDF":
        print("This is pdf")   
        ConvertPdf(i)
        resumes.extend([ConvertPdf(i)])
    if Temp[1]=="doc" or Temp[1]=="Doc" or Temp[1]=="DOC" or Temp[1]=="docx" or Temp[1]=="Docx" or Temp[1]=="DOCX":
        print("This is word file")
        ConvertWord(i)
        resumes.extend([ConvertWord(i)])
print("###### DONE PARSING ######") 

This is LIST OF FILES
['PWC_Olivia Peter_Regulatory Manager.pdf', 'PWC_Penny Lim_Risk AM.pdf', 'Jam.docx']
######## PARSING #########
This is pdf
This is pdf


This is word file
###### DONE PARSING ######


In [5]:
#keywords
import spacy
from spacy.matcher import PhraseMatcher
import en_core_web_sm
import pandas as pd
from io import StringIO
from collections import Counter
nlp=spacy.load('en_core_web_sm')

def KeywordsFilter(j):
    keyword_dict=pd.read_excel('C:/Users/DELL-LH2018/Desktop/intern/Job skills.xlsx',sheet_name='data scientist',header=None,names=['keywords of ds'])
    keyword_dict1=pd.read_excel('C:/Users/DELL-LH2018/Desktop/intern/Job skills.xlsx',sheet_name='general skills',header=None,names=['keywords of gs'])
    data_scientist_words=[nlp(text) for text in keyword_dict['keywords of ds'].dropna(axis = 0)]
    general_skill_words=[nlp(text) for text in keyword_dict1['keywords of gs'].dropna(axis = 0)]
    matcher = PhraseMatcher(nlp.vocab)
    matcher.add('DS',None,*data_scientist_words)
    matcher.add('General',None,*general_skill_words)
    doc=nlp(j)
    d=[]
    matches=matcher(doc)
    filtered_matches=[]
    for m in sorted(matches,key=lambda m:(m[1],-m[2])):
        if not filtered_matches or (m[1]>=filtered_matches[-1][-1] or m[2]>filtered_matches[-1][-1]):
            filtered_matches.append(m)
            filtered_matches
    for match_id, start, end in filtered_matches:
        rule_id = nlp.vocab.strings[match_id]
        span = doc[start : end]
        d.append((rule_id, span.text))      
    keywords = "\n".join(f'{a[0]} {a[1]} ({b})' for a,b in Counter(d).items())
    df = pd.read_csv(StringIO(keywords),names = ['Keywords_List'])
    df1 = pd.DataFrame(df.Keywords_List.str.split(' ',1).tolist(),columns = ['Subject','Keyword'])
    df2 = pd.DataFrame(df1.Keyword.str.split('(',1).tolist(),columns = ['Keyword', 'Count'])
    df3 = pd.concat([df1['Subject'],df2['Keyword'], df2['Count']], axis =1) 
    df3['Count'] = df3['Count'].apply(lambda x: x.rstrip(")"))
    name = Temp[0]
    cname = pd.read_csv(StringIO(name),names = ['Candidate Name'])
    dataf = pd.concat([cname['Candidate Name'], df3['Subject'], df3['Keyword'], df3['Count']], axis = 1)
    dataf['Candidate Name'].fillna(dataf['Candidate Name'].iloc[0], inplace = True)
    return dataf

KeywordsFilter(str(resumes))


,Candidate Name,Subject,Keyword,Count
0,Jam,General,education,4
1,Jam,General,risk,5
2,Jam,DS,assurance,5
3,Jam,General,governance,1
4,Jam,General,regulatory compliance,1
...,...,...,...,...
124,Jam,General,integrity,1
125,Jam,General,general ledger,1
126,Jam,General,media,1
127,Jam,General,sales,1


In [42]:
#experience
import spacy
from spacy.matcher import PhraseMatcher
import en_core_web_sm
import pandas as pd
from io import StringIO
from collections import Counter
import numpy as np
nlp=spacy.load('en_core_web_sm')

def ExperienceFilter(k):
    keyword_dict2=pd.read_excel('C:/Users/DELL-LH2018/Desktop/intern/Job skills.xlsx',sheet_name='experience',header=None,names=['keywords of exp'])
    experience_words=[nlp(text) for text in keyword_dict2['keywords of exp'].dropna(axis = 0)]
    matcher = PhraseMatcher(nlp.vocab)
    matcher.add('Experience',None,*experience_words)
    doc=nlp(k)
    d=[]
    matches=matcher(doc)
    filtered_matches=[]
    for m in sorted(matches,key=lambda m:(m[1],-m[2])):
        if not filtered_matches or (m[1]>=filtered_matches[-1][-1] or m[2]>filtered_matches[-1][-1]):
            filtered_matches.append(m)
            filtered_matches
    for match_id, start, end in filtered_matches:
        rule_id = nlp.vocab.strings[match_id]
        span = doc[start : end]
        d.append((rule_id, span.text))      
    keywords = "\n".join(f'{a[0]} {a[1]} ({b})' for a,b in Counter(d).items())
    df = pd.read_csv(StringIO(keywords),names = ['Keywords_List'])
    df1 = pd.DataFrame(df.Keywords_List.str.split(' ',1).tolist(),columns = ['Subject','Keyword'])
    df2 = pd.DataFrame(df1.Keyword.str.split('(',1).tolist(),columns = ['Keyword', 'Count'])
    df3 = pd.concat([df1['Subject'],df2['Keyword'], df2['Count']], axis =1) 
    df3['Count'] = df3['Count'].apply(lambda x: x.rstrip(")"))
    name = Temp[0]
    cname = pd.read_csv(StringIO(name),names = ['Candidate Name'])
    datae = pd.concat([cname['Candidate Name'], df3['Subject'], df3['Keyword'], df3['Count']], axis = 1)
    datae['Candidate Name'].fillna(datae['Candidate Name'].iloc[0], inplace = True)
    return datae

ExperienceFilter(str(resumes))

,Candidate Name,Subject,Keyword,Count
0,Jam,Experience,analyst,1
1,Jam,Experience,business analyst,1
2,Jam,Experience,finance analyst,2


In [6]:
#skill API:
import requests

url = "https://api.promptapi.com/skills?q=python"
payload = {}
headers= {
  "apikey": "PRC9IcUURAUQL9qAvywXoJTUVztfsHRp"
}
response = requests.request("GET", url, headers=headers,data =payload)
status_code = response.status_code
result = response.text
result

'["Python", "Pythonect", "Python 1.5", "Python 2.4", "Python/C API", "Python HOWTOs", "PythonAnywhere", "Python Debugger", "Python Robotics", "Python Compilers"]\n'

In [7]:
#weight of every keywords:

In [43]:
#whole
import os
import glob
import pandas as pd
import numpy as np
    
list_of_files=[]
list_of_files_pdf=[]
list_of_files_doc=[]
list_of_files_docx=[]
resumes=[]
os.chdir('C:/Users/DELL-LH2018/Desktop/intern/All original resume')
for file in glob.glob('**/*.pdf',recursive=True):
    list_of_files_pdf.append(file)
for file in glob.glob('**/*.doc',recursive=True):
    list_of_files_doc.append(file)
for file in glob.glob('**/*.docx',recursive=True):
    list_of_files_docx.append(file)
list_of_files = list_of_files_pdf+list_of_files_doc+list_of_files_docx
print("This is LIST OF FILES")
print(list_of_files)
    
#convert
print("######## PARSING #########")
for i in enumerate(list_of_files):
    Temp= i[1].split(".")
    if Temp[1]=="pdf" or Temp[1]=="Pdf" or Temp[1]=="PDF":
        print("This is pdf")   
        print(KeywordsFilter(ConvertPdf(i)))
        print(ExperienceFilter(ConvertPdf(i)))
        
        kw=KeywordsFilter(ConvertPdf(i))
        sum1=np.sum(kw['Subject'] == 'DS')
        sum2=np.sum(kw['Subject'] == 'General')
        score1=0.5*sum1+0.5*sum2
        exp=ExperienceFilter(ConvertPdf(i))
        exp['Count']=pd.to_numeric(exp['Count'],errors='coerce')
        score2=np.sum(exp['Count'])
        print("The total score of",Temp[0],"is",score1,"+",score2)
    
        resumes.extend([ConvertPdf(i)])
    if Temp[1]=="doc" or Temp[1]=="Doc" or Temp[1]=="DOC" or Temp[1]=="docx" or Temp[1]=="Docx" or Temp[1]=="DOCX":
        print("This is word file")
        print(KeywordsFilter(ConvertWord(i)))
        print(ExperienceFilter(ConvertWord(i)))
        
        kw=KeywordsFilter(ConvertWord(i))
        sum1=np.sum(kw['Subject'] == 'DS')
        sum2=np.sum(kw['Subject'] == 'General')
        score1=0.5*sum1+0.5*sum2
        exp=ExperienceFilter(ConvertWord(i))
        exp['Count']=pd.to_numeric(exp['Count'],errors='coerce')
        score2=np.sum(exp['Count'])
        print("The total score of",Temp[0],"is",score1,"+",score2)
        
        resumes.extend([ConvertWord(i)])
print("###### DONE PARSING ######") 

This is LIST OF FILES
['PWC_Olivia Peter_Regulatory Manager.pdf', 'PWC_Penny Lim_Risk AM.pdf', 'Jam.docx']
######## PARSING #########
This is pdf


                        Candidate Name  Subject     Keyword Count
0  PWC_Olivia Peter_Regulatory Manager  General  education      1


                        Candidate Name Subject Keyword Count
0  PWC_Olivia Peter_Regulatory Manager     NaN     NaN   NaN


The total score of PWC_Olivia Peter_Regulatory Manager is 0.5 + 0.0
This is pdf
           Candidate Name  Subject                 Keyword Count
0   PWC_Penny Lim_Risk AM  General                   risk      5
1   PWC_Penny Lim_Risk AM       DS              assurance      5
2   PWC_Penny Lim_Risk AM  General             governance      1
3   PWC_Penny Lim_Risk AM  General  regulatory compliance      1
4   PWC_Penny Lim_Risk AM  General             compliance      3
..                    ...      ...                     ...   ...
69  PWC_Penny Lim_Risk AM  General          business case      1
70  PWC_Penny Lim_Risk AM  General                written      1
71  PWC_Penny Lim_Risk AM  General               mandarin      1
72  PWC_Penny Lim_Risk AM  General                    llc      1
73  PWC_Penny Lim_Risk AM  General          mental health      1

[74 rows x 4 columns]
          Candidate Name     Subject   Keyword Count
0  PWC_Penny Lim_Risk AM  Experience  analyst      1
The total s

In [5]:
#date
import spacy
import re

nlp = spacy.load("en_core_web_sm")
doc = nlp(str(resumes))

expression = (r'(?P<year>\d\d\d\d)'
                 r'(?:(?P<dsep>-|)'
                 r'(?:(?P<julian>\d\d\d)'
                 r'|(?P<month>\d\d)(?:(?P=dsep)(?P<day>\d\d))?))?')
for match in re.finditer(expression, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end)
    # This is a Span object or None if match doesn't map to valid token sequence
    if span is not None:
        print("Found match:", span.text)

Found match: 9171
Found match: 6979
Found match: 2014
Found match: 2012
Found match: 2014
Found match: 2011
Found match: 2012
Found match: 2011
Found match: 2011
Found match: 2010
Found match: 2010
Found match: 2015
Found match: 2008
Found match: 2012
Found match: 2010
Found match: 2010
Found match: 2012
Found match: 2012
Found match: 2010
Found match: 2009
Found match: 2009
Found match: 2012
Found match: 2009
Found match: 2011
Found match: 2008
Found match: 2008
Found match: 2007
Found match: 2007
Found match: 2015
Found match: 2013
Found match: 2014
Found match: 2015
Found match: 2011
Found match: 2016
Found match: 2016
Found match: 2016
Found match: 2010
Found match: 2013
Found match: 2013
Found match: 2008
Found match: 2008
Found match: 2005
